In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
import time
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics
from sklearn.metrics import RocCurveDisplay, roc_auc_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler


from tensorflow import keras
import tensorflow as tf

2023-07-17 16:13:41.918120: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-17 16:13:42.216006: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 16:13:43.995339: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# 리눅스 한글 설정
plt.rc('font', family = 'NanumGothic')

In [4]:
df = pd.read_pickle('mywafer.pkl')

In [5]:
df_train = df[df['train_test_label']=='Training']
df_test = df[df['train_test_label']=='Test']
df_real = df[df['train_test_label'].isna()]

In [6]:
X_tr = df_train['wafermap_reshape'].apply(lambda x: np.array(x))

In [7]:
X_te = df_test['wafermap_reshape'].apply(lambda x: np.array(x))

In [8]:
X_test = np.array([x for x in X_te])

In [9]:
X_train = np.array([x for x in X_tr])

In [10]:
y_train = df_train['failure_type'].astype('category').cat.codes
y_test = df_test['failure_type'].astype('category').cat.codes

### CNN_model 불러오기

In [12]:
m_cnn55 = tf.keras.models.load_model('model_CNN.h5')
m_cnn33 = tf.keras.models.load_model('model_CNN33.h5')

In [13]:
m_cnn55.evaluate(X_test, y_test)

2023-07-16 15:39:33.416366: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902


  35/3702 [..............................] - ETA: 17s - loss: 0.3054 - accuracy: 0.9536  

2023-07-16 15:39:35.227085: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3702/3702 [==============================] - 20s 4ms/step - loss: 1.0204 - accuracy: 0.7480


[1.0203837156295776, 0.7480011582374573]

In [14]:
m_cnn33.evaluate(X_test, y_test)

3702/3702 [==============================] - 15s 4ms/step - loss: 1.2765 - accuracy: 0.7116


[1.2764838933944702, 0.7115792036056519]

## CNN모델은 kernel size가 (5,5)인것이 (3,3)인것 보다 좋은 정확도를 보인다.
### kernel size가 크면 좋은 성능을 보일까? ==> (10,10)으로 다시 만들어보자

In [19]:
# CNN kernel_size를 (10,10)으로 설정
model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=(10,10), activation = 'relu', input_shape=(80, 80, 1),
                       padding = 'valid', data_format = 'channels_last'), # 2D는 평면
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Conv2D(32, kernel_size = (10,10), activation='relu'),
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    #keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(9)
])


model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
         optimizer = keras.optimizers.Adam(learning_rate = 0.001),
         metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 71, 71, 32)        3232      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 35, 35, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        102432    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               6

In [20]:
model.fit(X_train, y_train, epochs = 10)

Epoch 1/10


2023-07-16 15:01:02.447169: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x558a3198a3c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-16 15:01:02.447214: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2023-07-16 15:01:02.501586: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-16 15:01:02.896749: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1654/1654 [==============================] - 21s 10ms/step - loss: 0.2988 - accuracy: 0.9178
Epoch 2/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.1283 - accuracy: 0.9619
Epoch 3/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0890 - accuracy: 0.9725
Epoch 4/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0698 - accuracy: 0.9787
Epoch 5/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0541 - accuracy: 0.9830
Epoch 6/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0435 - accuracy: 0.9862
Epoch 7/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0364 - accuracy: 0.9888
Epoch 8/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0293 - accuracy: 0.9905
Epoch 9/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0222 - accuracy: 0.9927
Epoch 10/10
1654/1654 [==============================] - 16s 10ms/step - lo

In [ ]:
model.save('model_CNN10.h5')

In [16]:
m_cnn10 = tf.keras.models.load_model('model_CNN10.h5')

In [17]:
m_cnn10.evaluate(X_test, y_test)

3702/3702 [==============================] - 17s 5ms/step - loss: 1.2267 - accuracy: 0.5837


[1.2266993522644043, 0.5836886167526245]

### 10,10으로 했을 때도 5,5보다 좋지 못한 성능을 보인다. kernel size를 10과 5사이에서 조율이 필요하다.

In [31]:
model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=(8,8), activation = 'relu', input_shape=(80, 80, 1),
                       padding = 'valid', data_format = 'channels_last'), # 2D는 평면
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Conv2D(32, kernel_size = (8,8), activation='relu'),
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    #keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(9)
])


model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
         optimizer = keras.optimizers.Adam(learning_rate = 0.001),
         metrics = ['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 73, 73, 32)        2080      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 32)        65568     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

In [32]:
model.fit(X_train, y_train, epochs = 10)

Epoch 1/10
1654/1654 [==============================] - 19s 9ms/step - loss: 0.2756 - accuracy: 0.9242
Epoch 2/10
1654/1654 [==============================] - 16s 9ms/step - loss: 0.1173 - accuracy: 0.9643
Epoch 3/10
1654/1654 [==============================] - 16s 9ms/step - loss: 0.0836 - accuracy: 0.9738
Epoch 4/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0675 - accuracy: 0.9790
Epoch 5/10
1654/1654 [==============================] - 16s 9ms/step - loss: 0.0523 - accuracy: 0.9831
Epoch 6/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0400 - accuracy: 0.9873
Epoch 7/10
1654/1654 [==============================] - 15s 9ms/step - loss: 0.0312 - accuracy: 0.9902
Epoch 8/10
1654/1654 [==============================] - 15s 9ms/step - loss: 0.0266 - accuracy: 0.9916
Epoch 9/10
1654/1654 [==============================] - 15s 9ms/step - loss: 0.0226 - accuracy: 0.9927
Epoch 10/10
1654/1654 [==============================] - 15s 9ms/step -

In [33]:
model.save('model_CNN8.h5')

In [24]:
m_cnn8 = tf.keras.models.load_model('model_CNN8.h5')

In [25]:
m_cnn8.evaluate(X_test, y_test)

3702/3702 [==============================] - 17s 4ms/step - loss: 1.0979 - accuracy: 0.7371


[1.097948670387268, 0.7370594143867493]

### kernel size는 5,5가 가장 높은 정확도를 보여준다. 이제 5,5모델의 learning_rate를 바꿔보자

In [36]:
#learning_rate = 0.01

model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=(5,5), activation = 'relu', input_shape=(80, 80, 1),
                       padding = 'valid', data_format = 'channels_last'), # 2D는 평면
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Conv2D(32, kernel_size = (5,5), activation='relu'),
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    #keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(9)
])


model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
         optimizer = keras.optimizers.Adam(learning_rate = 0.01),
         metrics = ['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 76, 76, 32)        832       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 38, 38, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 34, 34, 32)        25632     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 17, 17, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 9248)              0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [37]:
model.fit(X_train, y_train, epochs = 10)

Epoch 1/10
1654/1654 [==============================] - 17s 9ms/step - loss: 0.3598 - accuracy: 0.9050
Epoch 2/10
1654/1654 [==============================] - 14s 9ms/step - loss: 0.2724 - accuracy: 0.9274
Epoch 3/10
1654/1654 [==============================] - 14s 9ms/step - loss: 0.2500 - accuracy: 0.9326
Epoch 4/10
1654/1654 [==============================] - 14s 9ms/step - loss: 0.2352 - accuracy: 0.9357
Epoch 5/10
1654/1654 [==============================] - 14s 9ms/step - loss: 0.2245 - accuracy: 0.9387
Epoch 6/10
1654/1654 [==============================] - 14s 9ms/step - loss: 0.2166 - accuracy: 0.9418
Epoch 7/10
1654/1654 [==============================] - 14s 9ms/step - loss: 0.2099 - accuracy: 0.9420
Epoch 8/10
1654/1654 [==============================] - 14s 9ms/step - loss: 0.2022 - accuracy: 0.9451
Epoch 9/10
1654/1654 [==============================] - 14s 9ms/step - loss: 0.1892 - accuracy: 0.9480
Epoch 10/10
1654/1654 [==============================] - 14s 9ms/step - l

In [38]:
model.save('m_cnn5_rate001.h5')

In [29]:
m_cnn5_rate001 = tf.keras.models.load_model('m_cnn5_rate001.h5')

In [30]:
m_cnn5_rate001.evaluate(X_test, y_test)

3702/3702 [==============================] - 16s 4ms/step - loss: 5.8369 - accuracy: 0.4293


[5.836947441101074, 0.4293469488620758]

### learning_rate 0.01은 매우 낮은 정확도를 보인다.

In [37]:
#learning_rate = 0.0001
model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=(5,5), activation = 'relu', input_shape=(80, 80, 1),
                       padding = 'valid', data_format = 'channels_last'), # 2D는 평면
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Conv2D(32, kernel_size = (5,5), activation='relu'),
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    #keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(9)
])


model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
         optimizer = keras.optimizers.Adam(learning_rate = 0.0001),
         metrics = ['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 76, 76, 32)        832       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 38, 38, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 34, 34, 32)        25632     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 17, 17, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 9248)              0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [38]:
model.fit(X_test, y_test, epochs = 10)

Epoch 1/10


2023-07-16 15:52:07.407648: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x556312dc6d60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-16 15:52:07.407694: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2023-07-16 15:52:07.474351: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-16 15:52:07.981290: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3702/3702 [==============================] - 37s 9ms/step - loss: 0.3127 - accuracy: 0.9350
Epoch 2/10
3702/3702 [==============================] - 32s 9ms/step - loss: 0.2258 - accuracy: 0.9425
Epoch 3/10
3702/3702 [==============================] - 32s 9ms/step - loss: 0.1887 - accuracy: 0.9500
Epoch 4/10
3702/3702 [==============================] - 32s 9ms/step - loss: 0.1687 - accuracy: 0.9534
Epoch 5/10
3702/3702 [==============================] - 32s 9ms/step - loss: 0.1531 - accuracy: 0.9566
Epoch 6/10
3702/3702 [==============================] - 32s 9ms/step - loss: 0.1399 - accuracy: 0.9590
Epoch 7/10
3702/3702 [==============================] - 32s 9ms/step - loss: 0.1287 - accuracy: 0.9610
Epoch 8/10
3702/3702 [==============================] - 32s 9ms/step - loss: 0.1202 - accuracy: 0.9625
Epoch 9/10
3702/3702 [==============================] - 32s 9ms/step - loss: 0.1135 - accuracy: 0.9641
Epoch 10/10
3702/3702 [==============================] - 32s 9ms/step - loss: 0.1072

In [41]:
model.save('model_cnn5_rate.h5')

In [42]:
m_cnn5_rate = tf.keras.models.load_model('model_cnn5_rate.h5')

In [44]:
m_cnn5_rate.evaluate(X_test, y_test)

3702/3702 [==============================] - 16s 4ms/step - loss: 0.0963 - accuracy: 0.9693


[0.09626504778862, 0.9692515730857849]

In [12]:
#learning_rate = 0.0001
model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=(5,5), activation = 'relu', input_shape=(80, 80, 1),
                       padding = 'valid', data_format = 'channels_last'), # 2D는 평면
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Conv2D(32, kernel_size = (5,5), activation='relu'),
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    #keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(9)
])


model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
         optimizer = keras.optimizers.Adam(learning_rate = 0.00001),
         metrics = ['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 76, 76, 32)        832       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 38, 38, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 34, 34, 32)        25632     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 17, 17, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 9248)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

In [13]:
model.fit(X_test, y_test, epochs = 10)

Epoch 1/10


2023-07-17 16:14:48.498771: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-07-17 16:14:50.416938: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-17 16:14:50.440436: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x55963a05db60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-17 16:14:50.440480: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2023-07-17 16:14:50.484921: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-17 16:14:50.867461: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lif

3702/3702 [==============================] - 39s 8ms/step - loss: 0.3971 - accuracy: 0.9310
Epoch 2/10
3702/3702 [==============================] - 30s 8ms/step - loss: 0.3201 - accuracy: 0.9344
Epoch 3/10
3702/3702 [==============================] - 30s 8ms/step - loss: 0.2969 - accuracy: 0.9361
Epoch 4/10
3702/3702 [==============================] - 30s 8ms/step - loss: 0.2779 - accuracy: 0.9366
Epoch 5/10
3702/3702 [==============================] - 30s 8ms/step - loss: 0.2618 - accuracy: 0.9375
Epoch 6/10
3702/3702 [==============================] - 30s 8ms/step - loss: 0.2482 - accuracy: 0.9387
Epoch 7/10
3702/3702 [==============================] - 30s 8ms/step - loss: 0.2375 - accuracy: 0.9401
Epoch 8/10
3702/3702 [==============================] - 30s 8ms/step - loss: 0.2283 - accuracy: 0.9413
Epoch 9/10
3702/3702 [==============================] - 30s 8ms/step - loss: 0.2203 - accuracy: 0.9431
Epoch 10/10
3702/3702 [==============================] - 30s 8ms/step - loss: 0.2131

In [15]:
model.save('m_cnn5_rate2.h5')

In [16]:
cnn_rate2 = tf.keras.models.load_model('m_cnn5_rate2.h5')

In [17]:
cnn_rate2.evaluate(X_test, y_test)

3702/3702 [==============================] - 16s 4ms/step - loss: 0.2070 - accuracy: 0.9448


[0.20704811811447144, 0.944843590259552]

### 딥러닝 CNN 모델 중 kernel size 는 (5,5), learning_rate는 0.0001(10e-4)모델이 가장 이상적인 모델이다.